```
import datetime
import pandas as pd

import pandas.io.sql as pdsql
import psycopg2

from pmutils import creds
from pmutils.query import run_sql

rs_conn = creds.create_connection()

rs_conn
```

```
ip_data = '''
    select
        uc.signup_ip_state_code as state_code,
        count(du.user_id) as users,
        si.pop_2019,
        sum(case when du.buyer_activated_at is not null then 1 else 0 end) as buyers,
        cast(cast(count(du.user_id) as float)/cast(si.pop_2019 as float) as dec(13,4)) as perc_total_pop
    from analytics.dw_users as du
    join analytics.dw_users_cs as uc
        on du.user_id = uc.user_id
    join analytics_scratch.ag_state_info as si
        on uc.signup_ip_state_code = state_code
    where
        du.home_domain = 'us'
        and uc.signup_ip is not null
        --and du.gender in ('male', 'male (hidden)')
    group by uc.signup_ip_state_code,
        si.pop_2019
'''

ip_data = pdsql.read_sql_query(ip_data,con=rs_conn)
```

```
import plotly.plotly as py 
import plotly.graph_objs as go 
import pandas as pd 
  
# some more libraries to plot graph 
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot 
  
# To establish connection 
init_notebook_mode(connected = True) 

# plot geographical plots 
data = dict(type = 'choropleth', 
  
            # location: Arizoana, California, Newyork 
            locations = locs, 
              
            # States of USA 
            locationmode = 'USA-states', 
              
            # colorscale can be added as per requirement 
            colorscale = 'Viridis', 
              
            # text can be given anything you like 
            # text = ['text 1', 'text 2', 'text 3'], 
            z = perc_total_pop, 
            colorbar = {'title': '% Total Pop'}) 
              
layout = dict(geo ={'scope': 'usa'}) 
  
# passing data dictionary as a list  
choromap = go.Figure(data = [data], layout = layout) 
  
# plotting graph 
iplot(choromap)
```